### 🔹 Импортируются три ключевые библиотеки:  
	•	numpy — для числовых вычислений;  
	•	pandas — для табличной обработки данных;  
	•	datasets из Hugging Face — позволяет загружать готовые датасеты.  

In [1]:
# 📦 Импорт необходимых библиотек
import numpy as np
import pandas as pd
from datasets import load_dataset

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 🔹 Загружается датасет emotion — это датасет текстов с метками эмоций (joy, anger, fear, и т.д.) с помощью Hugging Face.

In [2]:
# 📚 Загрузка открытого датасета (пример: emotion классификация)
dataset = load_dataset("dair-ai/emotion")

Generating test split: 100%|██████████| 2000/2000 [00:00<00:00, 665551.25 examples/s]


### 🔹 Печатается первая строка обучающего набора (train), чтобы увидеть, как устроены записи — обычно это словарь с полями text и label.

In [3]:
# 👁️ Быстрый взгляд на структуру train-набора
print("Пример строки из датасета:")
print(dataset['train'][0])

Пример строки из датасета:
{'text': 'i didnt feel humiliated', 'label': 0}


### 🔹 Преобразуем HuggingFace Dataset в pandas.DataFrame — так проще анализировать и обрабатывать данные таблично.

In [4]:
# 🔄 Преобразование HuggingFace Dataset в DataFrame для удобной работы
df = dataset['train'].to_pandas()

### 🔹 Считаем, сколько записей приходится на каждый класс (эмоцию). Это важно для анализа сбалансированности выборки.

In [5]:
# 📊 Анализ распределения классов
print("\nРаспределение по классам эмоций:")
print(df['label'].value_counts())


Распределение по классам эмоций:
label
1    5362
0    4666
3    2159
4    1937
2    1304
5     572
Name: count, dtype: int64


### 🔹 Метки в датасете закодированы как числа. Здесь мы получаем список названий классов и заменяем числа на текст (например, 2 → joy).

In [6]:
# 🔄 Преобразование числовых меток в текстовые (сопоставим с оригинальными)
label_names = dataset['train'].features['label'].names
df['emotion'] = df['label'].apply(lambda x: label_names[x])

### 🔹 Текст приводится к нижнему регистру и очищается от пунктуации и спецсимволов с помощью регулярного выражения.

In [7]:
# 🧹 Пример простой очистки текста
df['clean_text'] = df['text'].str.lower().str.replace(r"[^\w\s]", "", regex=True)

### 🔹 Создаются новые признаки:
	•	text_length — количество слов в тексте;
	•	char_count — количество символов;
	•	avg_word_len — средняя длина слова.
Добавляется маленькое значение 1e-5, чтобы избежать деления на ноль.

In [8]:
# ➕ Создание числовых признаков с помощью NumPy
df['text_length'] = df['clean_text'].apply(lambda x: len(x.split()))
df['char_count'] = df['clean_text'].apply(len)
df['avg_word_len'] = df['char_count'] / (df['text_length'] + 1e-5)

### 🔹 Группируем данные по эмоциям и считаем среднюю длину текста. Это помогает понять особенности данных по каждой категории.

In [9]:
# 🧠 Пример базового анализа
print("\nСредняя длина текста по эмоциям:")
print(df.groupby("emotion")['text_length'].mean())


Средняя длина текста по эмоциям:
emotion
anger       19.229736
fear        18.844605
joy         19.498135
love        20.700153
sadness     18.361980
surprise    19.970280
Name: text_length, dtype: float64


### 🔹 Фильтруются записи, где текст состоит менее чем из 5 слов — может быть полезно для последующей очистки или анализа аномалий.

In [10]:
# ✅ Пример выборки с фильтрацией
short_texts = df[df['text_length'] < 5]
print(f"\nКоличество коротких текстов (<5 слов): {len(short_texts)}")


Количество коротких текстов (<5 слов): 363


### 💡 Итого:
Этот код — базовый, но эффективный пример обработки текстовых данных и подготовки к обучению или анализу. Он демонстрирует:
	•	Работу с реальными датасетами
	•	Владение библиотеками pandas, datasets, numpy
	•	Применение простых методов EDA (Exploratory Data Analysis)